In [1]:
D.<x> = PolynomialRing(QQ)
R = PolynomialRing(QQ,'x')
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x

def H4(n,m):
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
    #if argument is an S type then 'polynomial' takes no argument.

def H4strike(n,m): 
    poly=H4(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer


def H6(n,m):
    def base6(n,m):
        jay=J(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(jay-1))
        return L((-1)^m*(num/den)).O(n+1)
    return S(base6(n,m)^(1/(m-2))).polynomial()
     #if argument is an S type then 'polynomial' takes no argument.

def H6strike(n,m): 
    poly=H6(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return ((ddstep1(n,m)-ddstep2(n,m))/1728)

def deltaDaggerStrike(n,m): 
    poly=deltaDagger(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def deltaDagger2(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))

# def deltaDaggerStrike(n,m):
 #   return S(((H4strike(n,3)^3-H6strike(n,3)^2)/1728)).truncate(n+1)
    
def deltaDagger2Strike(n,m): 
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def first(list):return list[0]
def last(list):return list[-1]

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

print("ok")

ok


In [ ]:
#from 'very long H4 12sept22', Output snipped.

data=[]
import time
start=time.time()/60
for m in [3..1203]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no6.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

data=[]
import time
start=time.time()/60
for m in [1204..2403]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no7.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

data=[]
import time
start=time.time()/60
for m in [2404..3610]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no9.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

data=[]
import time
start=time.time()/60
for m in [3611..4818]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no10.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

In [ ]:
#from 'very long H6 12sept22', Output snipped.

data=[]
import time
start=time.time()/60
for m in [3..1203]:
    poly=H6(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no5.txt','wb') # H6 series
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

data=[]
import time
start=time.time()/60
for m in [1204..2403]:
    poly=H6(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no8.txt','wb') # H6 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

data=[]
import time
start=time.time()/60
for m in [2404..3610]:
    poly=H6(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no11.txt','wb') # H6 series
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

data=[]
import time
start=time.time()/60
for m in [3611..4818]:
    poly=H6(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no12.txt','wb') # H6 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

In [ ]:
# this failed so I do it again.
#printing snipped.
data=[]
import time
start=time.time()/60
for m in [3611..4818]:
    poly=H6(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no12.txt','wb') # H6 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

In [ ]:
# printing snipped.
import pickle
r41 = open('/Users/barrybrent/12sept22no6.txt','rb')  # H4 series 
rs41=pickle.load(r41)
r41.close()
s41=stripQuotationMarks(rs41)

r61 = open('/Users/barrybrent/12sept22no5.txt','rb') # H6 series
rs61=pickle.load(r61)
r61.close()
s61=stripQuotationMarks(rs61)


import time
w1 = open('/Users/barrybrent/13sept22no2.txt','wb') # 1st deltaDagger2Strike series
data=[]
start = time.time()
for n in [0..1200]:
    start = time.time()
    m=s41[n][0]
    h6squared=s61[n][1]^2
    h4cubed=s41[n][1]^3
    dagger=D((h4cubed-h6squared)).truncate(100)
    poly=dagger
    polylist=poly.list()
    polydegree=poly.degree()
    answer=D(sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree]))
    data=data+[[m,answer]]
    finish = time.time()
    print([m,answer.degree(),finish-start])
t = pickle.dumps(str(data))
w1.write(t)
w1.close()
print(len(t))

In [3]:
import pickle
rfile =open('/Users/barrybrent/13sept22no2.txt','rb') # 1st deltaDagger2Strike series
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

1201


In [ ]:
# printing snipped
import pickle
r42 = open('/Users/barrybrent/12sept22no7.txt','rb') # H4 series 
rs42=pickle.load(r42)
r42.close()
s42=stripQuotationMarks(rs42)

r62 = open('/Users/barrybrent/12sept22no8.txt','rb') # H6 series 
rs62=pickle.load(r62)
r62.close()
s62=stripQuotationMarks(rs62)


import time
w1 = open('/Users/barrybrent/14sept22no1.txt','wb') # 2nd deltaDagger2Strike series
data=[]
start = time.time()
for n in [0..1199]:
    start = time.time()
    m=s42[n][0]
    h6squared=s62[n][1]^2
    h4cubed=s42[n][1]^3
    dagger=D((h4cubed-h6squared)).truncate(100)
    poly=dagger
    polylist=poly.list()
    polydegree=poly.degree()
    answer=D(sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree]))
    data=data+[[m,answer]]
    finish = time.time()
    print([m,answer.degree(),finish-start])
t = pickle.dumps(str(data))
w1.write(t)
w1.close()
print(len(t))

In [ ]:
# printning snipped.
import pickle
r43 =  open('/Users/barrybrent/12sept22no9.txt','rb')
rs43=pickle.load(r43)
r43.close()
s43=stripQuotationMarks(rs43)

r63 = open('/Users/barrybrent/12sept22no11.txt','rb') # H6 series
rs63=pickle.load(r63)
r63.close()
s63=stripQuotationMarks(rs63)


import time
w1 = open('/Users/barrybrent/14sept22no2.txt','wb') # 3rd deltaDagger2Strike series
data=[]
start = time.time()
for n in [0..1206]:
    start = time.time()
    m=s43[n][0]
    h6squared=s63[n][1]^2
    h4cubed=s43[n][1]^3
    dagger=D((h4cubed-h6squared)).truncate(100)
    poly=dagger
    polylist=poly.list()
    polydegree=poly.degree()
    answer=D(sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree]))
    data=data+[[m,answer]]
    finish = time.time()
    print([m,answer.degree(),finish-start])
t = pickle.dumps(str(data))
w1.write(t)
w1.close()

In [ ]:
# printing snipped.
import pickle
r44 =open('/Users/barrybrent/12sept22no10.txt','rb') # H4 series 
rs44=pickle.load(r44)
r44.close()
s44=stripQuotationMarks(rs44)

r64 = open('/Users/barrybrent/12sept22no12.txt','rb') # H6 series 
rs64=pickle.load(r64)
r64.close()
s64=stripQuotationMarks(rs64)

import time
w1 = open('/Users/barrybrent/14sept22no3.txt','wb') # 4th deltaDagger2Strike series
data=[]
start = time.time()
for n in [0..1207]:
    start = time.time()
    m=s44[n][0]
    h6squared=s64[n][1]^2
    h4cubed=s44[n][1]^3
    dagger=D((h4cubed-h6squared)).truncate(100)
    poly=dagger
    polylist=poly.list()
    polydegree=poly.degree()
    answer=D(sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree]))
    data=data+[[m,answer]]
    finish = time.time()
    print([m,answer.degree(),finish-start])
t = pickle.dumps(str(data))
w1.write(t)
w1.close()

In [6]:
import pickle
rfile =open('/Users/barrybrent/14sept22no3.txt','rb') # 4th deltaDagger2Strike series
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

1208


In [ ]:
# interpolating polynomials
import pickle
r1 =open('/Users/barrybrent/13sept22no2.txt','rb') # 1st deltaDagger2Strike series
r1s=pickle.load(r1)
r1.close()
s1=stripQuotationMarks(r1s)

r2 = open('/Users/barrybrent/14sept22no1.txt','rb') # 2nd deltaDagger2Strike series
r2s=pickle.load(r2)
r2.close()
s2=stripQuotationMarks(r2s)

r3 = open('/Users/barrybrent/14sept22no2.txt','rb') # 3rd deltaDagger2Strike series
r3s=pickle.load(r3)
r3.close()
s3=stripQuotationMarks(r3s)

r4 =open('/Users/barrybrent/14sept22no3.txt','rb') # 4th deltaDagger2Strike series
r4s=pickle.load(r4)
r4.close()
s4=stripQuotationMarks(r4s)

print(len(s1[0][1]),len(s2[0][1]),len(s3[0][1]),len(s4[0][1]))
s=s1+s2+s3+s4 
data=[]
for k in [0..len(s)-1]:

In [8]:
poly=D(3*x^2+9*x-12)
print(numericalfactor(poly))

3


In [10]:
# interpolating polynomials
import pickle
# interpolating polynomials
import pickle
r1 =open('/Users/barrybrent/13sept22no2.txt','rb') # 1st deltaDagger2Strike series
r1s=pickle.load(r1)
r1.close()
s1=stripQuotationMarks(r1s)

r2 = open('/Users/barrybrent/14sept22no1.txt','rb') # 2nd deltaDagger2Strike series
r2s=pickle.load(r2)
r2.close()
s2=stripQuotationMarks(r2s)

r3 = open('/Users/barrybrent/14sept22no2.txt','rb') # 3rd deltaDagger2Strike series
r3s=pickle.load(r3)
r3.close()
s3=stripQuotationMarks(r3s)

r4 =open('/Users/barrybrent/14sept22no3.txt','rb') # 4th deltaDagger2Strike series
r4s=pickle.load(r4)
r4.close()
s4=stripQuotationMarks(r4s)

s=s1+s2+s3+s4 

polydata=[]
import time
wfile = open('/Users/barrybrent/22sept22no1.txt','wb') #deltaDagger2Strike series polynomials
start = time.time()/60
for n in [1..98]:
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf]]
        
    rl=R.lagrange_polynomial(data)
    
    polydata=polydata+[[n,rl]]
    finish = time.time()/60
    print([n,rl.degree(),finish-start])
    

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() 

[1, 3, 0.4809606671333313]
[2, 6, 0.9531788863241673]
[3, 9, 1.4141996502876282]
[4, 12, 1.8750401511788368]
[5, 15, 2.343214001506567]
[6, 18, 2.8003487698733807]
[7, 21, 3.2602456845343113]
[8, 24, 3.72314215824008]
[9, 27, 4.202639102935791]
[10, 30, 4.6622284688055515]
[11, 33, 5.139477390795946]
[12, 36, 5.628117337822914]
[13, 39, 6.1156871020793915]
[14, 42, 6.595281034708023]
[15, 45, 7.080014102160931]
[16, 48, 7.560368821024895]
[17, 51, 8.022533718496561]
[18, 54, 8.49972827360034]
[19, 57, 8.973056808114052]
[20, 60, 9.445529270917177]
[21, 63, 9.911315888166428]
[22, 66, 10.380909454077482]
[23, 69, 10.866236705332994]
[24, 72, 11.338698904961348]
[25, 75, 11.813794735819101]
[26, 78, 12.293761506676674]
[27, 81, 12.782377507537603]
[28, 84, 13.269068703055382]
[29, 87, 13.74960258975625]
[30, 90, 14.237119536846876]
[31, 93, 14.728098902851343]
[32, 96, 15.240626469254494]
[33, 99, 15.729457341134548]
[34, 102, 16.216806937009096]
[35, 105, 16.69967832416296]
[36, 108, 17

In [5]:
poly=D(x^5+3*x^4-2*x^3+11*x-17).truncate(3)
print(poly)

11*x - 17


In [5]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
print(s[0])
poly=D(s[0][1])
print(poly.factor())
print(numericalfactor(poly))

[1, 48*x^3 + 16*x^2 + 96*x]
(48) * x * (x^2 + 1/3*x + 2)
48


In [6]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
print(len(s))

98


In [9]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    data=data+[nf/48]
print(data)

[1, -13, 28, -109, 126, -364, 344, -877, 757, -1638, 1332, -3052, 2198, -4472, 3528, -7021, 4914, -9841, 6860, -13734, 9632, -17316, 12168, -24556, 15751, -28574, 20440, -37496, 24390, -45864, 29792, -56173, 37296, -63882, 43344, -82513, 50654, -89180, 61544, -110502, 68922, -125216, 79508, -145188, 95382, -158184, 103824, -196588, 117993, -204763, 137592, -239582, 148878, -265720, 167832, -301688, 192080, -317070, 205380, -384552, 226982, -387296, 260408, -449389, 276948, -484848, 300764, -535626, 340704, -563472, 357912, -663889, 389018, -658502, 441028, -747740, 458208, -800072, 493040, -884646, 551881, -895986, 571788, -1049888, 619164, -1033604, 682920, -1168164, 704970, -1239966, 756112, -1326312, 834176, -1349712, 864360, -1572844, 912674, -1533909]


In [37]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
data=[]
data2=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    if mod(n,2)==0:
        poly=D(s[k][1])
        nf=numericalfactor(poly)
        data=data+[(n,ZZ(mod(nf,sigma(n,3)))/6)]
print(data)
# here we're looking at even n, but
#alternate entries here agree with nf for odd n (see below.) Thus
# if n = 2 mod 4, nf(n) 

[(2, 1), (4, 4), (6, 28), (8, 4), (10, 126), (12, 112), (14, 344), (16, 4), (18, 757), (20, 504), (22, 1332), (24, 112), (26, 2198), (28, 1376), (30, 3528), (32, 4), (34, 4914), (36, 3028), (38, 6860), (40, 504), (42, 9632), (44, 5328), (46, 12168), (48, 112), (50, 15751), (52, 8792), (54, 20440), (56, 1376), (58, 24390), (60, 14112), (62, 29792), (64, 4), (66, 37296), (68, 19656), (70, 43344), (72, 3028), (74, 50654), (76, 27440), (78, 61544), (80, 504), (82, 68922), (84, 38528), (86, 79508), (88, 5328), (90, 95382), (92, 48672), (94, 103824), (96, 112), (98, 117993)]


In [36]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
evens=[]
odds=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    if mod(n,2)==0:evens=evens+[nf/48]
    if mod(n,2)==1:odds=odds+[nf/48]
print(evens)
print("==========================================================================================")
print(odds)

[-13, -109, -364, -877, -1638, -3052, -4472, -7021, -9841, -13734, -17316, -24556, -28574, -37496, -45864, -56173, -63882, -82513, -89180, -110502, -125216, -145188, -158184, -196588, -204763, -239582, -265720, -301688, -317070, -384552, -387296, -449389, -484848, -535626, -563472, -663889, -658502, -747740, -800072, -884646, -895986, -1049888, -1033604, -1168164, -1239966, -1326312, -1349712, -1572844, -1533909]
[1, 28, 126, 344, 757, 1332, 2198, 3528, 4914, 6860, 9632, 12168, 15751, 20440, 24390, 29792, 37296, 43344, 50654, 61544, 68922, 79508, 95382, 103824, 117993, 137592, 148878, 167832, 192080, 205380, 226982, 260408, 276948, 300764, 340704, 357912, 389018, 441028, 458208, 493040, 551881, 571788, 619164, 682920, 704970, 756112, 834176, 864360, 912674]


In [39]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
data=[]
data2=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    if mod(n,2)==1:
        poly=D(s[k][1])
        nf=numericalfactor(poly)
        data=data+[nf-48*sigma(n,3)]
print(data)
# found this pattern in OEIS page A045823

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [89]:
divisors(3)

[1, 3]

In [90]:
def sigmaOdd(n,k):
    answer=0
    dv=divisors(n)
    for j in [0..len(dv)-1]:
        d=dv[j]
        if mod(d,2)==1:answer=answer+d^k
    return answer
        
    

In [92]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
data2=[]
data3=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    if (mod(n,8)==0):
        poly=D(s[k][1])
        nf=-numericalfactor(poly) #note sign
        sn3=sigma(n/2,3)
        mns=ZZ(mod(nf,sn3))
        f=floor(QQ(nf/sn3))
        data2=data2+[f]
        data3=data3+[mns/48-sigmaOdd(n/8,3)]
print(data2);print();print(data3)
# data3 = OEIS page A051000: 2nd entry = sum cubes odd divisors of first entry

[576, 576, 576, 576, 576, 576, 576, 576, 576, 576, 576, 576]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [106]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
data2=[]
data3=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    if (mod(n,8)==4):
        poly=D(s[k][1])
        nf=-numericalfactor(poly) #note sign
        sn3=sigma(n/2,3)
        mns=ZZ(mod(nf,sn3))
        f=floor(QQ(nf/sn3))
        data2=data2+[f]
        data3=data3+[mns/3-sigma((n-4)/4+1,3)]
print(data2);print();print(data3)
# OEIS page A045823: 	a(n) = sigma_3(2*n+1)

[581, 581, 581, 581, 581, 581, 581, 581, 581, 581, 581, 581]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [70]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
data2=[]
data3=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    if (mod(n,8)==2):
        poly=D(s[k][1])
        nf=-numericalfactor(poly) #note sign
        sn3=sigma(n/2,3)
        mns=ZZ(mod(nf,sn3))
        f=floor(QQ(nf/sn3))
        data2=data2+[f]
        data3=data3+[mns]
print(data2);print();print(data3)

[624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [69]:
import pickle
r=open('/Users/barrybrent/22sept22no1.txt','rb')  #deltaDagger2Strike series polynomials
rs=pickle.load(r)
r.close()
s=stripQuotationMarks(rs)
data2=[]
data3=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    if (mod(n,8)==6):
        poly=D(s[k][1])
        nf=-numericalfactor(poly) #note sign
        sn3=sigma(n/2,3)
        mns=ZZ(mod(nf,sn3))
        f=floor(QQ(nf/sn3))
        data2=data2+[f]
        data3=data3+[mns]
print(data2);print();print(data3)

[624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
